In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
path = '/Users/noahraegrant/workspace/github.com/nohrg/DS5220/Final Project/MURA-v1.1'

#train
train_images = pd.DataFrame(pd.read_csv(path + '/train_image_paths.csv', header = None, names =['image_path']))
train_studies = pd.DataFrame(pd.read_csv(path + '/train_labeled_studies.csv', header= None, names=['study_path', 'label']))

#validation
val_images = pd.DataFrame(pd.read_csv(path + '/valid_image_paths.csv', header = None, names =['image_path']))
val_studies = pd.DataFrame(pd.read_csv(path + '/valid_labeled_studies.csv', header= None, names=['study_path', 'label']))

In [3]:
train_images['study_label'] = train_images['image_path'].str.rsplit('/', n=2, expand=True)[1].str.rsplit('_', n=1, expand=True)[1]

val_images['study_label'] = val_images['image_path'].str.rsplit('/', n=2, expand=True)[1].str.rsplit('_', n=1, expand=True)[1]
train_images['label'] = train_images['study_label'].map({'positive': 1, 'negative': 0})

val_images['label'] = val_images['study_label'].map({'positive': 1, 'negative': 0})
train_images['XR'] = train_images['image_path'].str.rsplit('/', n=5, expand=True)[2]
val_images['XR'] = val_images['image_path'].str.rsplit('/', n=5, expand=True)[2]

## Training

In [4]:
prefix = '/Users/noahraegrant/workspace/github.com/nohrg/DS5220/Final Project/'
train_images['image_path'] = prefix + train_images['image_path']
val_images['image_path'] = prefix + val_images['image_path']


In [5]:
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 pixels
    transforms.ToTensor(),  # Convert to tensor
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize (for pre-trained models)
])

# Custom Dataset class to handle loading and transformations
class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        label = self.df.iloc[idx]['label']

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
def create_dataloader(images:pd.DataFrame, transformer:transforms.Compose, batch_size:int):
    dataset = ImageDataset(images, transform=transformer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [7]:
train_loader = create_dataloader(train_images, transform, batch_size=64)
test_loader = create_dataloader(val_images, transform, batch_size=64)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
class UnifiedCNN(nn.Module):
    def __init__(self):
        super(UnifiedCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.batch1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.batch2 = nn.BatchNorm2d(256)
        self.fc = nn.Sequential(
            nn.Linear(in_features = 802816, out_features = 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features = 32, out_features= 1)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [10]:
model = UnifiedCNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

UnifiedCNN(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (batch1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (batch2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Sequential(
    (0): Linear(in_features=802816, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [11]:
import time
import numpy as np
def train_model(model, train_dataloader,val_dataloader, criterion, optimizer, num_epochs=10):
    # Xin's implementation of train_model
    train_loss, train_acc, val_loss, val_acc = [], [], [], []
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()  # Set the model to training mode
        running_loss = 0.0
        preds, labels = [], []
        for images, label in iter(train_dataloader):
            images, label = images.to(device), label.to(device).float()
            
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images).squeeze(1)
           
            loss = criterion(outputs, label)
            # Accumulate the loss
            running_loss += loss.item()
      

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            probabilities = torch.sigmoid(outputs)
            pred = (probabilities > 0.5).long() 
            preds.append(pred.detach().cpu().numpy())     
            labels.append(label.detach().cpu().numpy())
             
        # train_acc    
        preds = np.hstack(preds)
        labels = np.hstack(labels)
        train_acc.append(accuracy_score(labels, preds))
        
        # train_loss
        loss = running_loss / len(train_dataloader)
        
        train_loss.append(loss)
        
        # Perform validation
        model.eval()
        with torch.no_grad():
            preds, targets = [], []
            running_loss = 0
            for images, label in iter(val_dataloader):
                images, label = images.to(device), label.to(device).float()

                outputs = model(images).squeeze(1)

                loss = criterion(outputs, label)
                running_loss += loss.item()

                # Calculate val_acc
                probabilities = torch.sigmoid(outputs)
                pred = (probabilities > 0.5).long()  # For binary classification
                preds.append(pred.detach().cpu().numpy())
                targets.append(label.detach().cpu().numpy())
        
            preds = np.hstack(preds)
            targets = np.hstack(targets)
        
            val_acc.append(accuracy_score(targets, preds))
            val_loss.append(running_loss / len(val_dataloader))

        spent_time = time.time() - start_time

        print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss[-1]:.4f}, Validation Loss: {val_loss[-1]:.4f}. Spent {spent_time:.4f}s.")
        
    print("Training complete.")
    return train_loss, train_acc, val_loss, val_acc


In [13]:
num_epochs = 10
train_loss, train_acc, val_loss, val_acc = train_model(
    model, train_loader, test_loader, criterion, optimizer, num_epochs=num_epochs)

KeyboardInterrupt: 